In [ ]:
from pyinjective.async_client import AsyncClient
from pyinjective.core.network import Network

import pandas as pd 


network = Network.local()
client = AsyncClient(network)


async def sub_order(client, account_address) -> None:
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]


async def order_list(client, subaccount_id,market_id) -> None:
    orders = await client.fetch_subaccount_orders_list(subaccount_id=subaccount_id, market_id=market_id)
    return orders['orders']


async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(client, account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(client, i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    #print(df)
    df = df[["orderSide","subaccountId","price","quantity"]]
    df["price"] = df["price"].astype(float)/1e6
    df["quantity"] = df["quantity"].astype(float)
    df["quantity_in_$"] = df["price"] * df["quantity"]
    df = df[["orderSide","subaccountId","price","quantity_in_$"]]
    return df

In [2]:
from functions import marketId_to_ticker

ticker = "TTI/USDT PERP"
address = "inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z"

market_id = await marketId_to_ticker(ticker)
print(market_id)
await subfinder(account_address=address,market_id=market_id[0])

['0x37d4e69a77cdca055615d55c088d5bdf1bc75801e2c00a925e0b2dae62b5d660']


,orderSide,subaccountId,price,quantity_in_$
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000036,463.31,57541.24876
1,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000036,469.86,58354.73256
2,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,441.85,375.57250
3,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,439.52,373.59200
4,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,436.95,371.40750
5,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,436.71,371.20350
6,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,431.81,367.03850
7,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,431.58,366.84300
8,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,426.68,362.67800
9,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,426.21,362.27850
